In [23]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from typing import Dict, List

# Step 1: Initialize the Ollama Model
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0  # Ensures deterministic output
)

In [24]:

# Step 2: Define a Tool
@tool
def get_weather(location: str) -> str:
    """Get the weather from a location."""
    return f"The weather in {location} is Sunny."

# Step 3: Bind the tool to the model
llm_with_tools = llm.bind_tools([get_weather])


In [25]:

# Step 4: Define a function to process tool calls manually
def execute_tool_calls(msg: AIMessage) -> List[Dict]:
    """Extracts tool calls from the AIMessage and invokes the appropriate tools."""
    tool_map = {"get_weather": get_weather}
    tool_calls = msg.tool_calls.copy()

    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        # Invoke the tool manually
        tool_output = tool_map[tool_name].invoke(tool_args)
        tool_call["output"] = tool_output

    return tool_calls


In [26]:
# Step 5: Create the processing chain
chain = llm_with_tools | execute_tool_calls

# Step 6: Invoke the chain and process tool responses
query = "What's the weather in San Francisco, CA?"
tool_results = chain.invoke(query)

# Print tool responses
for tool_call in tool_results:
    print(f"Tool Name: {tool_call['name']}")
    print(f"Input Args: {tool_call['args']}")
    print(f"Output: {tool_call['output']}")


Tool Name: get_weather
Input Args: {'location': 'San Francisco, CA'}
Output: The weather in San Francisco, CA is Sunny.


In [29]:
tool_results

[{'name': 'get_weather',
  'args': {'location': 'San Francisco, CA'},
  'id': 'e670e286-69f6-4710-8176-075955c40e95',
  'type': 'tool_call',
  'output': 'The weather in San Francisco, CA is Sunny.'}]

In [30]:
# Step 4: Define a function to process tool calls manually
def execute_tool_calls_str(msg: AIMessage) -> str:
    """Extracts tool calls from the AIMessage, invokes the appropriate tools, and returns a formatted response."""
    tool_map = {"get_weather": get_weather}
    tool_calls = msg.tool_calls.copy()

    # If no tool calls are present, return the plain response
    if not tool_calls:
        return msg.content

    tool_responses = []
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]

        # Invoke the tool manually
        if tool_name in tool_map:
            tool_output = tool_map[tool_name].invoke(tool_args)
            tool_responses.append(tool_output)

    # Combine tool outputs into a single string
    return "\n".join(tool_responses)

# Step 5: Create the processing chain with StrOutputParser
chain = llm_with_tools | execute_tool_calls_str | StrOutputParser()

# Step 6: Invoke the chain and parse the response
query = "What's the weather in San Francisco, CA?"
parsed_response = chain.invoke(query)

# Print the final parsed response
print("Parsed Response:", parsed_response)


Parsed Response: The weather in San Francisco, CA is Sunny.
